In [48]:
import pandas as pd
import numpy as np

# Load PeMS08.rel edge list
rel = pd.read_csv("../PeMS08/PeMS08.rel")

# Get all unique LibCity node IDs
libcity_nodes = pd.unique(rel[['origin_id', 'destination_id']].values.ravel())
libcity_nodes.sort()  # should be 0 to 169 for PeMS08

# Map node IDs to index (in case they're not ordered 0 to N-1)
libcity_id_map = {nid: idx for idx, nid in enumerate(libcity_nodes)}
N = len(libcity_nodes)

# Build LibCity adjacency matrix
A_lib = np.zeros((N, N))

for _, row in rel.iterrows():
    i = libcity_id_map[row['origin_id']]
    j = libcity_id_map[row['destination_id']]
    A_lib[i, j] = row['cost']  # or use row['cost'] if you want weighted
    A_lib[j,i] = row['cost']

print(f"A_lib shape: {A_lib.shape}")

A_lib shape: (170, 170)


In [4]:
A_real = np.load('ca_rn_adj.npy')  # shape should be like (1000, 1000)
print(f"Official adj shape: {A_real.shape}")

Official adj shape: (8600, 8600)


In [6]:
import pandas as pd

# Load the statewide metadata
meta_df = pd.read_csv("ca_meta.csv")

# Filter to District 8 only (PeMS08)
d8_df = meta_df[meta_df['District'] == 8].copy()
d8_df.reset_index(drop=True, inplace=True)

print(f"District 8 sensors: {len(d8_df)}")

District 8 sensors: 1022


In [9]:
import numpy as np

# Full matrix: shape = (num_sensors_total, num_sensors_total)
A_real = np.load("ca_rn_adj.npy")

# Get sensor indices of D8 in the full list
# Assume full sensor list is in same order as A_real
all_sensor_ids = meta_df['ID'].tolist()
sensor_id_to_index = {sid: i for i, sid in enumerate(all_sensor_ids)}

# Map District 8 sensor IDs to their row/col indices
d8_indices = [sensor_id_to_index[sid] for sid in d8_df['ID'] if sid in sensor_id_to_index]

# Build submatrix
A_d8 = A_real[np.ix_(d8_indices, d8_indices)]

print(f"District 8 subgraph shape: {A_d8.shape}")

District 8 subgraph shape: (1022, 1022)


In [16]:
import numpy as np
import random
from tqdm import tqdm

# Inputs
A_lib_bin = (A_lib > 0).astype(int)  # LibCity graph
A_d8_bin = (A_d8 > 0).astype(int)    # Real D8 subgraph

M = A_d8_bin.shape[0]  # number of real sensors in D8
N = A_lib_bin.shape[0]  # number of LibCity nodes = 170

sensor_pool = list(range(M))  # real sensor indices in District 8

best_score = -1
best_mapping = None

# Try multiple random starts
for trial in tqdm(range(1000)):
    candidate = random.sample(sensor_pool, N)
    A_real_sub = A_d8_bin[np.ix_(candidate, candidate)]
    score = np.sum(A_real_sub == A_lib_bin)

    if score > best_score:
        best_score = score
        best_mapping = candidate

print(f"\n✅ Best match score: {best_score} / {N*N}")


100%|██████████| 1000/1000 [00:00<00:00, 1968.83it/s]


✅ Best match score: 28182 / 28900


In [20]:

# Map matched real indices to sensor IDs from d8_df
matched_sensor_ids = [d8_df.iloc[i]['ID'] for i in best_mapping]

# Create LibCity ID → Real Sensor ID mapping
libcity_to_real = {
    lib_id: real_sensor_id
    for lib_id, real_sensor_id in enumerate(matched_sensor_ids)
}

# Merge with ca_meta.csv
meta_df = pd.read_csv("ca_meta.csv")

# ⚠️ Fix column name if it's "ID" instead of "sensor_id"
meta_df.rename(columns={"ID": "sensor_id"}, inplace=True)

coords_df = pd.DataFrame({
    "libcity_id": list(libcity_to_real.keys()),
    "sensor_id": list(libcity_to_real.values())
}).merge(meta_df, on="sensor_id", how="left")

coords_df.to_csv("libcity_pems08_mapped_coords_greedy.csv", index=False)
print(coords_df.head())

coords_df.to_csv("libcity_pems08_mapped_coords_greedy.csv", index=False)


   libcity_id  sensor_id        Lat         Lng  District          County  \
0           0     820001  34.028347 -117.574003         8  San Bernardino   
1           1     825570  34.065957 -117.485675         8  San Bernardino   
2           2     801259  34.078035 -117.623770         8  San Bernardino   
3           3     819056  33.967254 -117.034163         8       Riverside   
4           4     822133  35.383477 -115.861276         8  San Bernardino   

      Fwy  Lanes      Type Direction   ID2  
0  SR60-E      4  Mainline         E  5953  
1   I10-E      4  Mainline         E  5408  
2   I10-W      4  Mainline         W  5518  
3   I10-E      3  Mainline         E  5461  
4   I15-N      3  Mainline         N  5779  


In [21]:
coords_df

,libcity_id,sensor_id,Lat,Lng,District,County,Fwy,Lanes,Type,Direction,ID2
0,0,820001,34.028347,-117.574003,8,San Bernardino,SR60-E,4,Mainline,E,5953
1,1,825570,34.065957,-117.485675,8,San Bernardino,I10-E,4,Mainline,E,5408
2,2,801259,34.078035,-117.623770,8,San Bernardino,I10-W,4,Mainline,W,5518
3,3,819056,33.967254,-117.034163,8,Riverside,I10-E,3,Mainline,E,5461
4,4,822133,35.383477,-115.861276,8,San Bernardino,I15-N,3,Mainline,N,5779
...,...,...,...,...,...,...,...,...,...,...,...
165,165,819605,34.134533,-117.668512,8,San Bernardino,I210-W,4,Mainline,W,6191
166,166,825734,34.014115,-117.343500,8,Riverside,I215-S,3,Mainline,S,6377
167,167,826110,34.134666,-117.347002,8,San Bernardino,I210-E,3,Mainline,E,6174
168,168,817143,33.575299,-117.219606,8,Riverside,I15-N,3,Mainline,N,5654


In [27]:
lib_costs = A_lib[A_lib > 0]
print(f"LibCity cost stats:")
print(f"  min: {lib_costs.min():.2f}")
print(f"  mean: {lib_costs.mean():.2f}")
print(f"  max: {lib_costs.max():.2f}")

LibCity cost stats:
  min: 6.30
  mean: 319.99
  max: 3274.40


In [28]:
real_weights = A_d8[(A_d8 > 0) & (A_d8 < 1)]  # skip diagonals = 1.0
print(f"Caltrans weight stats:")
print(f"  min: {real_weights.min():.4f}")
print(f"  mean: {real_weights.mean():.4f}")
print(f"  max: {real_weights.max():.4f}")

Caltrans weight stats:
  min: 0.0100
  mean: 0.3498
  max: 1.0000


In [49]:
import numpy as np

mean_cost = A_lib[A_lib > 0].mean()
mean_real = A_d8[(A_d8 > 0) & (A_d8 < 1)].mean()

estimated_sigma = -mean_cost / np.log(mean_real)

print(f"Mean LibCity cost: {mean_cost:.2f}")
print(f"Mean Caltrans weight: {mean_real:.4f}")
print(f"Estimated σ: {estimated_sigma:.2f}")

Mean LibCity cost: 321.41
Mean Caltrans weight: 0.3498
Estimated σ: 305.96


In [53]:
# Make a copy to avoid modifying A_lib directly
A_lib_copy = A_lib.copy()

# Replace 0s (non-edges) with ∞
A_lib_copy[A_lib_copy == 0] = np.inf

# Now apply the Gaussian kernel
sigma = estimated_sigma  # or use a fixed one like 50–150
A_lib_weighted = np.exp(-A_lib_copy / sigma)

np.fill_diagonal(A_lib_weighted, 1.0)


In [54]:
A_lib_weighted

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.29684613, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.29684613, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.56922814,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.56922814, 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [32]:
lib_costs = A_lib[A_lib > 0]
print("LibCity costs:", lib_costs.min(), lib_costs.mean(), lib_costs.max())


LibCity costs: 6.3 319.9880866425992 3274.4


In [68]:
import numpy as np
import random
from tqdm import tqdm

M = A_d8.shape[0]         # total nodes in real District 8 graph
N = A_lib_weighted.shape[0]  # number of LibCity nodes = 170
sensor_pool = list(range(M))  # All candidate real nodes

best_score = float('inf')  # smaller is better
best_mapping = None

for trial in tqdm(range(5000)):  # more = better coverage
    candidate = random.sample(sensor_pool, N)
    A_real_sub = A_d8[np.ix_(candidate, candidate)]

    # Compute mean absolute error where either graph has a non-zero edge
    mask = (A_lib_weighted > 0) | (A_real_sub > 0)
    cost_diff = np.abs(A_lib_weighted - A_real_sub)
    score = np.sum(cost_diff[mask])

    if score < best_score:
        best_score = score
        best_mapping = candidate

print(f"\n✅ Best match score (lower is better): {best_score:.2f}")


100%|██████████| 5000/5000 [00:02<00:00, 1768.76it/s]


✅ Best match score (lower is better): 299.87


In [71]:
import numpy as np
import random
from tqdm import tqdm

# Assume:
# - A_lib_weighted is your LibCity 170x170 weighted matrix
# - A_d8 is your District 8 real adjacency matrix (MxM)
# - best_mapping is the 170-node index list from previous match
# - sensor_pool = list(range(M))

def compute_score(A_lib, A_real_sub):
    mask = (A_lib > 0) | (A_real_sub > 0)
    return np.sum(np.abs(A_lib - A_real_sub)[mask])

# Start with current best
current_mapping = best_mapping[:]
remaining = list(set(sensor_pool) - set(current_mapping))
current_subgraph = A_d8[np.ix_(current_mapping, current_mapping)]
current_score = compute_score(A_lib_weighted, current_subgraph)

print(f"Initial score before refinement: {current_score:.2f}")

# Attempt local swaps
improved = True
iteration = 0
max_iterations = 10

while improved and iteration < max_iterations:
    improved = False
    iteration += 1

    for i in range(len(current_mapping)):
        original_node = current_mapping[i]

        for candidate_node in random.sample(remaining, min(len(remaining), 170)):  # try up to 25 swaps per node
            trial_mapping = current_mapping[:]
            trial_mapping[i] = candidate_node

            trial_subgraph = A_d8[np.ix_(trial_mapping, trial_mapping)]
            trial_score = compute_score(A_lib_weighted, trial_subgraph)

            if trial_score < current_score:
                # Accept improvement
                remaining.append(original_node)
                remaining.remove(candidate_node)
                current_mapping[i] = candidate_node
                current_score = trial_score
                improved = True
                best_mapping = current_mapping
                #break  # try next i

    print(f"Iteration {iteration} score: {current_score:.2f}")

print(f"\n✅ Final refined score: {current_score:.2f}")


Initial score before refinement: 154.56
Iteration 1 score: 154.48
Iteration 2 score: 153.98
Iteration 3 score: 153.98

✅ Final refined score: 153.98


In [67]:
A_real_final_sub = A_d8[np.ix_(current_mapping, current_mapping)]

In [72]:
A_real_final_sub

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.28948516, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.14618068, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [74]:
import pandas as pd

# 1. Get the real sensor IDs from the matched row indices
sensor_ids = [d8_df.iloc[i]['ID'] for i in best_mapping]

# 2. Load statewide metadata with coordinates
meta_df = pd.read_csv("ca_meta.csv")
meta_df.rename(columns={"ID": "sensor_id"}, inplace=True)

# 3. Build DataFrame: LibCity ID → Sensor ID → lat/lon
mapping_df = pd.DataFrame({
    "libcity_id": list(range(len(sensor_ids))),
    "sensor_id": sensor_ids
}).merge(meta_df, on="sensor_id", how="left")

# 4. Save it
mapping_df.to_csv("pems08_libcity_to_real_coords.csv", index=False)
print(mapping_df.head())


   libcity_id  sensor_id        Lat         Lng  District          County  \
0           0     821298  33.926239 -116.926106         8       Riverside   
1           1     820994  34.781913 -117.130846         8  San Bernardino   
2           2     821071  34.782032 -117.131069         8  San Bernardino   
3           3     822961  33.924439 -116.693670         8       Riverside   
4           4     817788  33.815500 -117.512331         8       Riverside   

     Fwy  Lanes      Type Direction   ID2  
0  I10-W      4  Mainline         W  5603  
1  I15-N      3  Mainline         N  5763  
2  I15-S      3  Mainline         S  5907  
3  I10-E      4  Mainline         E  5479  
4  I15-S      3  Mainline         S  5835  


In [75]:
mapping_df

,libcity_id,sensor_id,Lat,Lng,District,County,Fwy,Lanes,Type,Direction,ID2
0,0,821298,33.926239,-116.926106,8,Riverside,I10-W,4,Mainline,W,5603
1,1,820994,34.781913,-117.130846,8,San Bernardino,I15-N,3,Mainline,N,5763
2,2,821071,34.782032,-117.131069,8,San Bernardino,I15-S,3,Mainline,S,5907
3,3,822961,33.924439,-116.693670,8,Riverside,I10-E,4,Mainline,E,5479
4,4,817788,33.815500,-117.512331,8,Riverside,I15-S,3,Mainline,S,5835
...,...,...,...,...,...,...,...,...,...,...,...
165,165,817377,33.641546,-117.283515,8,Riverside,I15-S,3,Mainline,S,5809
166,166,817744,33.522150,-117.162950,8,Riverside,I15-S,4,Mainline,S,5796
167,167,801346,34.030152,-117.686430,8,San Bernardino,SR60-E,4,Mainline,E,5937
168,168,809912,34.025900,-117.723013,8,San Bernardino,SR60-E,4,Mainline,E,5932


In [77]:
mapping_df['centers'] = mapping_df.apply(lambda row: [row['Lat'], row['Lng']], axis=1)


In [78]:
vec_field = [[0]*170 for _ in range(170)]
for index, row in mapping_df.iterrows():
    for index1, row1 in mapping_df.iterrows():
        delta_x = row1['centers'][0] - row['centers'][0]
        delta_y = row1['centers'][1] - row['centers'][1]
        vec_field[index][index1]= np.array([delta_x,delta_y])
        
vec_field

[[array([0., 0.]),
  array([ 0.855674, -0.20474 ]),
  array([ 0.855793, -0.204963]),
  array([-0.0018  ,  0.232436]),
  array([-0.110739, -0.586225]),
  array([ 0.046338, -0.622614]),
  array([-0.074705, -0.608358]),
  array([ 0.209545, -0.444496]),
  array([ 0.124483, -0.623155]),
  array([ 0.209921, -0.507736]),
  array([ 0.386873, -0.550514]),
  array([-0.035251, -0.628958]),
  array([-0.306685, -0.245217]),
  array([ 0.209834, -0.443083]),
  array([-0.099327, -0.599112]),
  array([-0.236377, -0.409629]),
  array([-2.42e-04, -1.20e-05]),
  array([ 0.090272, -0.815018]),
  array([-0.198411, -0.452   ]),
  array([-0.275646, -0.362963]),
  array([-0.262778, -0.374106]),
  array([-0.119418, -0.308246]),
  array([-0.207814, -0.437201]),
  array([0.990263, 0.14611 ]),
  array([-0.403933, -0.236509]),
  array([1.495155, 1.187094]),
  array([ 0.20798 , -0.735411]),
  array([ 0.208622, -0.70595 ]),
  array([-0.040305, -0.592287]),
  array([ 0.074886, -0.137571]),
  array([-0.312547, -0.33550

In [80]:
v_np = np.array(vec_field)
np.save('VF_PeMS08.npy', v_np)